In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv('filtered_ShubmanGill.csv')

In [4]:
dataset.drop(columns=['Unnamed: 0'],inplace=True)

In [5]:
dataset

,inns_number,bat,team,bowl,opposition,ball,ball_id,outs,dismissal,over,...,control,inns,1s,2s,3s,4s,5s,6s,bowler_dismissed?,dots
0,1.0,Shubman Gill,Kolkata Knight Riders,Billy Stanlake,Sunrisers Hyderabad,5.0,13.05,0,NaN,14.0,...,1.0,1,0,0,0,0,0,0,0,1
1,1.0,Shubman Gill,Kolkata Knight Riders,Rashid Khan,Sunrisers Hyderabad,4.0,15.04,0,NaN,16.0,...,1.0,1,0,0,0,0,0,0,0,1
2,1.0,Shubman Gill,Kolkata Knight Riders,Rashid Khan,Sunrisers Hyderabad,2.0,15.02,0,NaN,16.0,...,1.0,1,1,0,0,0,0,0,0,0
3,1.0,Shubman Gill,Kolkata Knight Riders,Siddarth Kaul,Sunrisers Hyderabad,5.0,14.05,0,NaN,15.0,...,1.0,1,1,0,0,0,0,0,0,0
4,1.0,Shubman Gill,Kolkata Knight Riders,Rashid Khan,Sunrisers Hyderabad,5.0,15.05,0,NaN,16.0,...,1.0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2429,1.0,Shubman Gill,Gujarat Titans,Shardul Thakur,Chennai Super Kings,2.0,15.02,0,NaN,16.0,...,0.0,1,0,0,0,0,0,0,0,1
2430,1.0,Shubman Gill,Gujarat Titans,Shardul Thakur,Chennai Super Kings,3.0,15.03,0,NaN,16.0,...,1.0,1,0,1,0,0,0,0,0,0
2431,1.0,Shubman Gill,Gujarat Titans,Shardul Thakur,Chennai Super Kings,4.0,15.04,0,NaN,16.0,...,1.0,1,1,0,0,0,0,0,0,0
2432,1.0,Shubman Gill,Gujarat Titans,Tushar Deshpande,Chennai Super Kings,1.0,14.01,0,NaN,15.0,...,1.0,1,1,0,0,0,0,0,0,0


In [12]:
dataset.cumsum()

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [7]:
dataset.columns

Index(['inns_number', 'bat', 'team', 'bowl', 'opposition', 'ball', 'ball_id',
       'outs', 'dismissal', 'over', 'phase', 'noball', 'wide', 'date', 'year',
       'ground', 'country', 'bat_hand', 'bowl_style', 'bowl_kind',
       'bowl_category', 'runs', 'balls', 'line', 'length', 'shot', 'control',
       'inns', '1s', '2s', '3s', '4s', '5s', '6s', 'bowler_dismissed?',
       'dots'],
      dtype='object')